### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    if not isinstance(text, str):  
        return text  
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [2]:
#a)
baby_df['cleaned_reviews'] = baby_df['review'].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [3]:
#b)

def test_cleaned_reviews(index):
    try:
        result = baby_df["cleaned_reviews"][index]
        if result == "":  
            print(f"Index {index}: Empty string ('')")
        else:
            print(f"Index {index}: {result}")
    except KeyError:
        print(f"Index {index} has been removed from the DataFrame.")


baby_df['cleaned_reviews'] = baby_df['cleaned_reviews'].fillna("")

#short test:
test_cleaned_reviews(37)
test_cleaned_reviews(38)
test_cleaned_reviews(39)
test_cleaned_reviews(58)
test_cleaned_reviews(59)
test_cleaned_reviews(60)

Index 37: Our baby loves this book  has loved it for a while now  He enjoys when I read it to him and enjoys playing with it on his own
Index 38: Empty string ('')
Index 39: My son likes brushing elmos teeth Almost too nice of a book to let the kids play with haha 
Index 58: Empty string ('')
Index 59: Absolutely love it and all of the Scripture in it  I purchased the Baby Boy version for my grandson when he was born and my daughterinlaw was thrilled to receive the same book again
Index 60: Bought as a gift for my daughter in law She loved it Simple but a place to put important memories down I loved the verses at bottom of pages


In [4]:
#c)
baby_df = baby_df[baby_df['rating'] != 3]

#short test:
sum(baby_df["rating"] == 3)

0

In [5]:
#d) 
baby_df.loc[baby_df['rating'] <= 2, 'rating'] = -1
baby_df.loc[baby_df['rating'] >= 4, 'rating'] = 1

#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [7]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [8]:
#a)
from sklearn.model_selection import train_test_split

train, test = train_test_split(baby_df.dropna(), test_size=0.2, random_state=42)

In [9]:
#b)
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(list(train["review"]))
y = train["rating"]
x_test = vectorizer.transform(list(test["review"]))
y_test = test["rating"]

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [17]:
#a)
model = LogisticRegression()
model.fit(x, y)


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
#b)
zipped_coef = list(zip(list(range(model.coef_.shape[1])), model.coef_[0]))
sorted_coef = sorted(zipped_coef, key= lambda v: v[1])
sorted_coef = np.array(sorted_coef)
sorted_coef_indexes = sorted_coef[:, 0].astype(int)
words = np.array(vectorizer.get_feature_names_out())
most_positive_words = words[sorted_coef_indexes[-10:]]
most_negative_words = words[sorted_coef_indexes[:10]]
print("most positive words: ", most_positive_words)
print("most negative words: ", most_negative_words)
#hint: model.coef_, vectorizer.get_feature_names()

most positive words:  ['helped' 'negative' 'complaints' 'pleased' 'amazing' 'awesome' 'perfect'
 'satisfied' 'excellent' 'worry']
most negative words:  ['disappointing' 'worst' 'poorly' 'concept' 'worthless' 'disappointed'
 'theory' 'terrible' 'returning' 'useless']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [13]:
#a)
model.predict(x_test)

array([ 1,  1, -1, ...,  1,  1,  1])

In [14]:
#b)
probabilty = model.predict_proba(x_test)
probabilty
#hint: model.predict_proba()

array([[3.11436193e-01, 6.88563807e-01],
       [4.58939132e-03, 9.95410609e-01],
       [8.30891227e-01, 1.69108773e-01],
       ...,
       [1.39895449e-01, 8.60104551e-01],
       [1.59636773e-02, 9.84036323e-01],
       [3.58822349e-04, 9.99641178e-01]])

In [15]:
#c) 
print("most positive: ", test.iloc[probabilty.T[1].argmax()]["review"])
print("\n most negative: ", test.iloc[probabilty.T[0].argmax()]["review"])
#hint: use the results of b)

most positive:  My husband and I assembled this Pack n' Play last night and so far we are very happy with it.  The assembly seemed a bit complicated but I think that's a normal reaction from a couple who has never put together baby gear before.  My husband was able to put it together in about 15-20 minutes and the entire setup seems very sturdy and secure.  The elevated bassinet is also 100% level (we checked with an actual level) and the changer is solid.  I was able to move the Pack n' Play easily and get it through our doors with no issues (it just fit) although I'm almost eight months along.  We haven't had a chance to try the mechanical portion yet as it appears you need to add your own batteries.My only surprise thus far was with the exact color.  The black looks great; I purposely was looking for simple modern colors.  However, I thought the trim color was a beige but it is actually a medium gray.  While I like the clean and simple black coloring of the main section, the gray ma

In [16]:
#d) 
model.score(x_test, y_test)

0.9356892805408016

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [18]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [19]:
#a)
vectorizer_small = CountVectorizer(vocabulary=significant_words)

x_small = vectorizer_small.fit_transform(list(train["review"]))
y_small = train["rating"]
x_test_small = vectorizer_small.transform(list(test["review"]))
y_test_small = test["rating"]

light_model = LogisticRegression()
light_model.fit(x_small, y_small)
proba = light_model.predict_proba(x_test_small)
print("most positive: ", test.iloc[proba.T[1].argmax()]["review"])
print("\nmost negative: ", test.iloc[proba.T[0].argmax()]["review"])
light_model.score(x_test_small, y_test_small)

most positive:  UPDATE 9/30/2012: Okay, so I've had a chance to use this stroller for months now, and I have to say I'm satisfied with it.1. I love the color orangie we picked. We recently went to a fair (The Big E) and my fianc&eacute; said it was easy to spot us in a crowd when he was returning with our food.2. The drink holders are very small, and if you have a large cup it's not going to fit because the handle bars that you push the stroller with gets in the way. A soda can fits perfectly though (just to give you an idea), and for the larger drinks, we actually unzipped the pockets you can put your cell phone or wallet in, and set it in there lol worked for us.3. Okay, I was a little disappointed with this basket. It is big compared to many other double strollers, but its not THAT BIG. My JJ Cole Mode diaper bag already takes up half the basket (but I have to lay it flat because it pokes at my kid's butt if not) and its not as easily accessible as I thought it would be. There is a 

0.8672440849830999

In [20]:
#b)
for word, coef in zip(vectorizer_small.get_feature_names_out(), light_model.coef_[0]):
    print(f"{word}: {coef}")

love: 1.3695639614376605
great: 0.9390286281836189
easy: 1.180621465787631
old: 0.10758407430492509
little: 0.48342960090866754
perfect: 1.4930511270436548
loves: 1.7184706805970134
well: 0.5185346245595523
able: 0.19325270019614244
car: 0.07071286667178822
broke: -1.6297400760546477
less: -0.1564738132835682
even: -0.515596242644489
waste: -1.9762772435552034
disappointed: -2.373613905849694
work: -0.6255981631729428
product: -0.31450121877494397
money: -0.9120965568185173
would: -0.34671853808878844
return: -2.0625096333076662


In [21]:
#c)
print(f"light model achieves {light_model.score(x_test_small, y_test_small)}")
print(f"standard model achieves {model.score(x_test, y_test)}")
#hint: %time, %timeit

light model achieves 0.8672440849830999
standard model achieves 0.9356892805408016


In [ ]:
import sys, time


CPU times: total: 0 ns
Wall time: 0 ns


array([-1,  1,  1, ...,  1,  1,  1])

In [27]:
%%time
%%timeit
light_model.predict(x_test_small)

843 μs ± 135 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CPU times: total: 6.62 s
Wall time: 6.95 s


In [26]:
%%time
%%timeit
model.predict(x_test)

9.72 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
CPU times: total: 7.78 s
Wall time: 8.04 s
